In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import datetime
from dateutil import relativedelta
from sklearn import preprocessing
from sklearn.cross_validation import cross_val_score

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
cnx = sqlite3.connect('database.sqlite')
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", cnx)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [3]:
PA = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)
print(PA.shape)
PA.head()

(183978, 42)


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [4]:
PA.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


In [5]:
Player = pd.read_sql_query("SELECT * FROM Player", cnx)
print(Player.shape)
Player.head()

(11060, 7)


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [6]:
Player.describe()

,id,player_api_id,player_fifa_api_id,height,weight
count,11060.000000,11060.000000,11060.000000,11060.000000,11060.000000
mean,5537.511392,156582.427215,165664.910488,181.867445,168.380289
std,3197.692647,160713.700624,58649.928360,6.369201,14.990217
min,1.000000,2625.000000,2.000000,157.480000,117.000000
25%,2767.750000,35555.500000,151889.500000,177.800000,159.000000
50%,5536.500000,96619.500000,184671.000000,182.880000,168.000000
75%,8306.250000,212470.500000,203883.250000,185.420000,179.000000
max,11075.000000,750584.000000,234141.000000,208.280000,243.000000


In [7]:
DS = pd.read_sql_query("SELECT * FROM Player_Attributes PA INNER JOIN Player P ON PA.player_api_id = P.player_api_id", cnx)
DS.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,gk_kicking,gk_positioning,gk_reflexes,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,10.0,8.0,8.0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,10.0,8.0,8.0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,10.0,8.0,8.0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,9.0,7.0,7.0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,9.0,7.0,7.0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187


In [8]:
DS['age'] = 1

def age(dob):
    dobasdt = datetime.strptime(dob, '%Y-%m-%d %H:%M:%S')
    today = datetime.now()
    difference = relativedelta.relativedelta(today, dobasdt)
    
    return difference.years

DS['age'] = DS.birthday.apply(age).astype('float64', copy=False)
DS = DS.dropna()
Y = DS.overall_rating

In [9]:
DS.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes', 'id', 'player_api_id', 'player_name',
       'player_fifa_api_id', 'birthday', 'height', 'weight', 'age'],
      dtype='object')

In [10]:
DS.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'player_name', 'birthday', 'overall_rating'], axis = 1, inplace=True)

In [11]:
DS.head()

,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,height,weight,age
0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,...,69.0,69.0,6.0,11.0,10.0,8.0,8.0,182.88,187,26.0
1,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,...,69.0,69.0,6.0,11.0,10.0,8.0,8.0,182.88,187,26.0
2,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,...,66.0,69.0,6.0,11.0,10.0,8.0,8.0,182.88,187,26.0
3,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,...,63.0,66.0,5.0,10.0,9.0,7.0,7.0,182.88,187,26.0
4,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,...,63.0,66.0,5.0,10.0,9.0,7.0,7.0,182.88,187,26.0


In [12]:
dummy_PF = pd.get_dummies(DS['preferred_foot'], prefix='preferred_foot')
dummy_PF = dummy_PF.astype('float64', copy=False)
dummy_PF.head()

,preferred_foot_left,preferred_foot_right
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [13]:
data = DS.join(dummy_PF.ix[:, 'preferred_foot_left':])
del data['preferred_foot']
DS = data
DS.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,potential,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,...,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,height,weight,age,preferred_foot_left,preferred_foot_right
0,71.0,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,...,6.0,11.0,10.0,8.0,8.0,182.88,187,26.0,0.0,1.0
1,71.0,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,...,6.0,11.0,10.0,8.0,8.0,182.88,187,26.0,0.0,1.0
2,66.0,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,...,6.0,11.0,10.0,8.0,8.0,182.88,187,26.0,0.0,1.0
3,65.0,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,...,5.0,10.0,9.0,7.0,7.0,182.88,187,26.0,0.0,1.0
4,65.0,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,...,5.0,10.0,9.0,7.0,7.0,182.88,187,26.0,0.0,1.0


In [14]:
dummy_AWR = pd.get_dummies(DS['attacking_work_rate'], prefix='attacking_work_rate')
dummy_AWR = dummy_AWR.astype('float64', copy=False)
dummy_AWR.head()

,attacking_work_rate_None,attacking_work_rate_high,attacking_work_rate_le,attacking_work_rate_low,attacking_work_rate_medium,attacking_work_rate_norm,attacking_work_rate_stoc,attacking_work_rate_y
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [15]:
data = DS.join(dummy_AWR.ix[:, 'attacking_work_rate_None':])
del data['attacking_work_rate']
DS = data
DS.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,potential,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,preferred_foot_left,preferred_foot_right,attacking_work_rate_None,attacking_work_rate_high,attacking_work_rate_le,attacking_work_rate_low,attacking_work_rate_medium,attacking_work_rate_norm,attacking_work_rate_stoc,attacking_work_rate_y
0,71.0,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,71.0,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,66.0,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,65.0,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,65.0,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
dummy_DWR = pd.get_dummies(DS['defensive_work_rate'], prefix='defensive_work_rate')
dummy_DWR = dummy_DWR.astype('float64', copy=False)
dummy_DWR.head()

,defensive_work_rate_0,defensive_work_rate_1,defensive_work_rate_2,defensive_work_rate_3,defensive_work_rate_4,defensive_work_rate_5,defensive_work_rate_6,defensive_work_rate_7,defensive_work_rate_8,defensive_work_rate_9,defensive_work_rate_ean,defensive_work_rate_es,defensive_work_rate_high,defensive_work_rate_low,defensive_work_rate_medium,defensive_work_rate_o,defensive_work_rate_ormal,defensive_work_rate_tocky
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [17]:
data = DS.join(dummy_DWR.ix[:, :])
del data['defensive_work_rate']
DS = data
DS.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,defensive_work_rate_8,defensive_work_rate_9,defensive_work_rate_ean,defensive_work_rate_es,defensive_work_rate_high,defensive_work_rate_low,defensive_work_rate_medium,defensive_work_rate_o,defensive_work_rate_ormal,defensive_work_rate_tocky
0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [18]:
lm = LinearRegression()
lm.fit(DS, Y)

print(lm.intercept_)
print(lm.coef_)

-12.0996711233
[  4.92853577e-01   2.28792347e-03   1.30586328e-02   5.23976413e-02
   5.83691773e-02  -5.09140195e-03   1.01231334e-02   1.06435493e-02
   7.36620287e-03   1.12210063e-02   1.08301527e-01   1.49240113e-02
   1.93456257e-02  -4.45297234e-03   1.66349699e-01  -5.38241323e-03
   1.20927676e-02   1.18598496e-02   6.62650939e-03   5.10547827e-02
  -7.99793135e-03   3.66261993e-03  -2.20820593e-03  -7.50205547e-03
  -2.13941057e-02   9.69214499e-04   2.44482529e-02   8.26167306e-03
  -1.59246104e-02   1.50026054e-01   2.51436199e-02  -5.13918978e-02
   3.98977423e-02   2.52402862e-02  -9.36340701e-03  -3.22130671e-03
   3.29735489e-01  -2.91914238e-03   2.91914238e-03   2.65618474e-01
   6.26582593e-01  -6.23484581e-01   1.35402362e+00   4.85462779e-01
  -2.49835260e-01  -1.02122782e+00  -8.37139795e-01   2.54459652e-01
   1.07774953e+00  -2.53982236e-01   1.77046958e-01  -3.20341817e-01
  -1.54096333e-01  -5.39288810e-02   7.77130224e-01   7.07500963e-01
   4.54405498e-01  

In [19]:
y_pred = lm.predict(DS)
print(mean_squared_error(Y, y_pred))

6.43290841751


In [20]:
X_train, X_test, y_train, y_test = train_test_split(DS, Y, test_size=0.25, random_state=123456)

lm = LinearRegression()
lm.fit(X_train, y_train)

print(lm.intercept_)
print(lm.coef_)

print("*" * 50)

y_pred = lm.predict(X_test)
print(mean_squared_error(y_test, y_pred))

-11.9697624129
[  4.92084431e-01   2.70357395e-03   1.29265143e-02   5.20356034e-02
   5.95925793e-02  -5.11567314e-03   1.00527387e-02   1.07987990e-02
   6.95056363e-03   1.12599669e-02   1.08862017e-01   1.54033971e-02
   1.95400027e-02  -5.78238758e-03   1.65449041e-01  -5.73293160e-03
   1.08734709e-02   1.26341480e-02   5.81138597e-03   5.13339562e-02
  -6.43079491e-03   4.25651912e-03  -1.59276380e-03  -8.18193333e-03
  -2.15413250e-02   1.24345953e-03   2.46010095e-02   7.24459459e-03
  -1.57726404e-02   1.50367353e-01   2.67555131e-02  -5.13913673e-02
   3.82950665e-02   2.53691412e-02  -1.02369175e-02  -2.86139567e-03
   3.28810060e-01  -3.69638865e-04   3.69638865e-04   2.96022500e-01
   6.23648868e-01  -7.26049325e-01   1.31380831e+00   4.82305086e-01
  -2.93026767e-01  -9.10046659e-01  -7.86662013e-01   1.10562965e-01
   9.56989443e-01  -2.17488189e-01   7.16708619e-02  -6.50827953e-01
  -6.30886336e-03  -1.40403280e-01   8.56536227e-01   1.03896248e+00
   4.59434212e-01  

In [21]:
scores = cross_val_score(LinearRegression(), DS, Y, scoring='neg_mean_squared_error', cv=10)
scores, scores.mean()

(array([-6.31201501, -6.34837443, -6.40924211, -6.71384647, -6.39495577,
        -6.83301381, -6.28275544, -6.5252498 , -6.48673828, -6.39866629]),
 -6.4704857408942669)